In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/cleaned_movies_30k.csv")
embeddings = np.load("../data/movie_embeddings.npy")

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

In [4]:
df[["title", "genres", "imdb_rating"]].head()

,title,genres,imdb_rating
0,Zootopia 2,"Animation, Comedy, Adventure, Family, Mystery",7.700000
1,The Running Man,"Action, Thriller, Science Fiction",6.700000
2,TRON: Ares,"Science Fiction, Adventure, Action",6.400000
3,Five Nights at Freddy's 2,"Horror, Thriller",6.100000
4,The Cult,Horror,6.201565


In [5]:
mood_map = {
    "happy": "feel good fun uplifting light hearted comedy friendship joy",
    "sad": "emotional sad heartbreak loss grief deep drama",
    "romantic": "love romance relationships emotional couple",
    "angry": "revenge anger rage intense action conflict",
    "relaxed": "calm peaceful slow beautiful emotional soothing",
    "thriller": "suspense mystery crime dark psychological",
    "motivational": "inspiring success struggle hope determination"
}


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_by_mood(mood, df, embeddings, model, top_n=5, alpha=0.7):
    mood = mood.lower().strip()

    if mood not in mood_map:
        return f"Available moods: {list(mood_map.keys())}"

    mood_text = mood_map[mood]
    mood_embedding = model.encode(mood_text).reshape(1, -1)

    similarity_scores = cosine_similarity(mood_embedding, embeddings)[0]
    imdb_norm = df["imdb_rating"] / 10

    final_score = alpha * similarity_scores + (1 - alpha) * imdb_norm

    result_df = df.copy()
    result_df["final_score"] = final_score

    recommendations = (
        result_df
        .sort_values("final_score", ascending=False)
        .head(top_n)
        .reset_index(drop=True)
    )

    recommendations.insert(0, "Rank", range(1, top_n + 1))

    return recommendations[["Rank", "title", "genres", "imdb_rating"]]

In [7]:
recommend_by_mood(
    mood="romantic",
    df=df,
    embeddings=embeddings,
    model=model
)

,Rank,title,genres,imdb_rating
0,1,Our Time,Romance,6.201565
1,2,Critical Eleven,"Drama, Romance",7.100000
2,3,"Yes, We're Open","Comedy, Romance",6.100000
3,4,Love Aaj Kal,"Comedy, Drama, Romance",6.800000
4,5,The Woman Next Door,"Drama, Romance",7.200000
